In [2]:
!pip install opencv-python-headless ipywidgets

import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import ipywidgets as widgets
from IPython.display import display, clear_output

uploader = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Upload Image'
)

threshold_slider = widgets.IntSlider(
    value=50, min=0, max=255, description='Threshold'
)

min_area_slider = widgets.IntSlider(
    value=20, min=1, max=1000, description='Min Spot Area (px)'
)

density_input = widgets.FloatText(
    value=8.96, description='True Density (g/cm³)'
)

process_button = widgets.Button(description='Process Image')
output = widgets.Output()

def on_process_clicked(b):
    with output:
        clear_output()
        if not uploader.value:
            print("⚠️ Please upload an image first.")
            return

        key = next(iter(uploader.value))
        data = uploader.value[key]['content']
        img_array = np.frombuffer(data, np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        thresh = threshold_slider.value
        _, pore_mask = cv2.threshold(
            gray, thresh, 255, cv2.THRESH_BINARY_INV
        )

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
        clean_mask = cv2.morphologyEx(
            pore_mask, cv2.MORPH_OPEN, kernel, iterations=2
        )

        contours, _ = cv2.findContours(
            clean_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )
        min_area = min_area_slider.value
        total_pore_area = sum(
            cv2.contourArea(c) for c in contours if cv2.contourArea(c) >= min_area
        )

        h, w = gray.shape
        porosity_frac = total_pore_area / (h * w)
        apparent_density = density_input.value * (1 - porosity_frac)

        for c in contours:
            if cv2.contourArea(c) >= min_area:
                cv2.drawContours(img, [c], -1, (0,0,255), 1)

        print(f"Image size: {w}×{h} px")
        print(f"Total pore area: {total_pore_area:.0f} px")
        print(f"Surface porosity: {porosity_frac*100:.2f} %")
        print(f"Apparent density: {apparent_density:.4f} g/cm³")
        cv2_imshow(img)

process_button.on_click(on_process_clicked)

# --- LAYOUT ---
ui = widgets.VBox([
    uploader,
    widgets.HBox([threshold_slider, min_area_slider]),
    density_input,
    process_button,
    output
])

display(ui)
